In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from sqlalchemy import create_engine
import re
import requests as req
import json
import datetime
import time

In [2]:
KEY = 'b50bc5-5af08a'

BCN = 'BCN'

status = 'landed'

url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={BCN}&type=departure&status={status}"


## Get flight information request

In [11]:
def get_data(airport,status = 'landed'):
    
    KEY = 'b50bc5-5af08a'
    url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={airport}&type=departure&status={status}"    
    response = req.get(url)
    print(response)
    print((datetime.datetime.now()))
    json_file = response.json()

    if type(json_file) == list:
        df_f = pd.DataFrame(json_file)
        return df_f

    
    
    
    
    

In [12]:
df_barcelona = get_data(airport='BCN')

<Response [200]>
2019-07-25 14:40:33.470605


In [36]:
type(df_barcelona)

pandas.core.frame.DataFrame

In [15]:
def clean_data(data_df):
    data_df["airln_name"] = data_df["airline"].apply(lambda x: x['name'])
    data_df["airln_code"] = data_df["airline"].apply(lambda x: x['iataCode'])
    data_df['arri_delay'] = data_df['arrival'].apply(lambda x: x['delay'])
    data_df['arri_airpt'] = data_df['arrival'].apply(lambda x: x['iataCode'])
    data_df['schd_arri_time'] = data_df['arrival'].apply(lambda x: x['scheduledTime'])
    data_df['codeshared'] = data_df['codeshared'].apply(lambda x: x['airline']['name'] if x else None)
    data_df['depart_delay'] = data_df['departure'].apply(lambda x: x['delay'])
    data_df['depart_airpt'] = data_df['departure'].apply(lambda x: x['iataCode'])
    data_df['schd_depart_time'] = data_df['departure'].apply(lambda x: x['scheduledTime'])
    data_df['depart_gate'] = data_df['departure'].apply(lambda x: x['gate'])
    data_df['depart_terminal'] = data_df['departure'].apply(lambda x: x['terminal'])
    data_df['icaonumb'] = data_df['flight'].apply(lambda x: x['icaoNumber'])
    data_df['flight'] = data_df['flight'].apply(lambda x: x['number'])
    data_df = data_df.drop(['airline','arrival','departure'], axis =1)
    
    return data_df
    

## Get airports by country

In [16]:
def get_data_airpts(country):
    
    KEY = 'b50bc5-5af08a'
    url_airpt = f'https://aviation-edge.com/v2/public/airportDatabase?key={KEY}&codeIso2Country={country}'
    
    response = req.get(url_airpt)
    print(response)
    print((datetime.datetime.now()))
    json_file = response.json()
    df = pd.DataFrame(json_file)
    return df

In [17]:
spain_airpts = get_data_airpts('ES')

<Response [200]>
2019-07-25 14:43:19.685737


In [115]:
spain_airpts.head(2)

,GMT,airportId,codeIataAirport,codeIataCity,codeIcaoAirport,codeIso2Country,geonameId,latitudeAirport,longitudeAirport,nameAirport,nameCountry,phone,timezone
0,1,27,ABC,ABC,LEAB,ES,6299323,38.948334,-1.863333,Los Llanos,Spain,,Europe/Madrid
1,0,55,ACE,ACE,GCRR,ES,6297189,28.950668,-13.609059,Lanzarote,Spain,902 404 704,Atlantic/Canary


In [101]:
def get_iata_codes(df_airpts):
    airpts_lst = df_airpts['codeIataAirport'].to_list()
    country_code = df_airpts['codeIso2Country'].to_list()
    name = df_airpts['nameAirport'].to_list()
    return airpts_lst,country_code, name

### Spain

In [102]:
spain_airpts_lst, country_code_sp, name_airport = get_iata_codes(df_airpts=spain_airpts)


In [89]:
print(spain_airpts_lst)

['ABC', 'ACE', 'AEI', 'AGP', 'ALC', 'BCN', 'BIO', 'BJZ', 'CDT', 'CDZ', 'EAS', 'EEL', 'EEM', 'EER', 'EEU', 'FES', 'FGR', 'FUE', 'GEN', 'GGA', 'GMZ', 'GRO', 'GRX', 'HEV', 'HSK', 'IBZ', 'ILD', 'JCU', 'LCG', 'LEI', 'LEN', 'LEU', 'LPA', 'MAD', 'MAH', 'MJV', 'MLN', 'NDO', 'ODB', 'OVD', 'OZI', 'OZL', 'OZP', 'OZU', 'PMI', 'PNA', 'PXS', 'QGN', 'QLQ', 'REU', 'RGS', 'RJL', 'RMU', 'ROZ', 'RRA', 'SCQ', 'SDR', 'SLM', 'SPC', 'SPO', 'SVQ', 'TFN', 'TFS', 'TOJ', 'UER', 'UTL', 'VDE', 'VGO', 'VIT', 'VLC', 'VLL', 'XIV', 'XJB', 'XJI', 'XJJ', 'XJN', 'XJO', 'XJR', 'XJU', 'XOC', 'XOJ', 'XOU', 'XQA', 'XRY', 'XTI', 'XTJ', 'XUA', 'XUF', 'XUT', 'XZZ', 'YJB', 'YJC', 'YJD', 'YJE', 'YJG', 'YJH', 'YJI', 'YJL', 'YJM', 'YJR', 'YJV', 'YJW', 'ZAZ', 'ZRC']


In [103]:
spain_airports = pd.DataFrame({'country_code':country_code_sp,
             'iata_code':spain_airpts_lst,
                              'name': name_airport})

In [104]:
spain_airports.to_sql('airports',engine, if_exists='append', index=False)

### France

In [22]:
france_airpts = get_data_airpts('FR')

<Response [200]>
2019-07-25 14:43:26.768482


In [105]:
france_airpts_lst,country_code_fr, name_airport = get_iata_codes(df_airpts=france_airpts)

In [106]:
france_airports = pd.DataFrame({'country_code':country_code_fr,
             'iata_code':france_airpts_lst,
                              'name': name_airport})

In [108]:
france_airports.to_sql('airports',engine, if_exists='append', index=False)

### Germany (DE)

In [24]:
germany_airpts = get_data_airpts('DE')

<Response [200]>
2019-07-25 14:43:29.140808


In [109]:
germany_airpts_lst,country_code_de, name_airport = get_iata_codes(df_airpts=germany_airpts)

In [110]:
germany_airports = pd.DataFrame({'country_code':country_code_de,
             'iata_code':germany_airpts_lst,
                              'name': name_airport})

In [111]:
germany_airports.to_sql('airports',engine, if_exists='append', index=False)

### Italy

In [26]:
italy_airpts = get_data_airpts('IT')

<Response [200]>
2019-07-25 14:43:31.721126


In [112]:
italy_airpts_lst, country_code_it, name_airport = get_iata_codes(df_airpts=italy_airpts)

In [113]:
italy_airports = pd.DataFrame({'country_code':country_code_it,
             'iata_code':italy_airpts_lst,
                              'name': name_airport})

In [114]:
italy_airports.to_sql('airports',engine, if_exists='append', index=False)

## All countries airports iata codes

In [29]:
countries_airpts_lst = spain_airpts_lst + france_airpts_lst + germany_airpts_lst + italy_airpts_lst

In [40]:
df_countries_airpt = pd.DataFrame(countries_airpts_lst)

## Wednesday, July 24

In [63]:
dict_airpt_df = {}

for aipt_code in countries_airpts_lst:
    
    df_airpt = get_data(airport = f"{aipt_code}")
    
    if type(df_airpt) == pd.core.frame.DataFrame:
        df_airpt_clean = clean_data(df_airpt)
        dict_airpt_df[aipt_code] = df_airpt_clean
        time.sleep(5)


<Response [200]>
2019-07-25 22:55:51.974490
<Response [200]>
2019-07-25 22:55:52.555895
<Response [200]>
2019-07-25 22:55:58.184680
<Response [200]>
2019-07-25 22:55:58.949581
<Response [200]>
2019-07-25 22:56:04.559957
<Response [200]>
2019-07-25 22:56:10.296826
<Response [200]>
2019-07-25 22:56:15.928780
<Response [200]>
2019-07-25 22:56:21.533599
<Response [200]>
2019-07-25 22:56:27.165543
<Response [200]>
2019-07-25 22:56:32.677500
<Response [200]>
2019-07-25 22:56:33.317224
<Response [200]>
2019-07-25 22:56:38.825310
<Response [200]>
2019-07-25 22:56:39.329395
<Response [200]>
2019-07-25 22:56:39.853182
<Response [200]>
2019-07-25 22:56:40.345492
<Response [200]>
2019-07-25 22:56:40.813567
<Response [200]>
2019-07-25 22:56:41.283409
<Response [200]>
2019-07-25 22:56:41.891423
<Response [200]>
2019-07-25 22:56:47.459681
<Response [200]>
2019-07-25 22:56:47.937741
<Response [200]>
2019-07-25 22:56:48.566737
<Response [200]>
2019-07-25 22:56:54.195062
<Response [200]>
2019-07-25 22:5

<Response [200]>
2019-07-25 23:02:55.141285
<Response [200]>
2019-07-25 23:02:55.999937
<Response [200]>
2019-07-25 23:03:01.800054
<Response [200]>
2019-07-25 23:03:02.517602
<Response [200]>
2019-07-25 23:03:03.179919
<Response [200]>
2019-07-25 23:03:09.015190
<Response [200]>
2019-07-25 23:03:09.617199
<Response [200]>
2019-07-25 23:03:10.269743
<Response [200]>
2019-07-25 23:03:10.934703
<Response [200]>
2019-07-25 23:03:16.580694
<Response [200]>
2019-07-25 23:03:22.085621
<Response [200]>
2019-07-25 23:03:22.757251
<Response [200]>
2019-07-25 23:03:28.690312
<Response [200]>
2019-07-25 23:03:29.280778
<Response [200]>
2019-07-25 23:03:29.951385
<Response [200]>
2019-07-25 23:03:36.189545
<Response [200]>
2019-07-25 23:03:36.821049
<Response [200]>
2019-07-25 23:03:37.419111
<Response [200]>
2019-07-25 23:03:38.520387
<Response [200]>
2019-07-25 23:03:39.327157
<Response [200]>
2019-07-25 23:03:40.128816
<Response [200]>
2019-07-25 23:03:41.421924
<Response [200]>
2019-07-25 23:0

<Response [200]>
2019-07-25 23:08:41.051639
<Response [200]>
2019-07-25 23:08:41.865854
<Response [200]>
2019-07-25 23:08:42.730020
<Response [200]>
2019-07-25 23:08:43.439710
<Response [200]>
2019-07-25 23:08:44.260017
<Response [200]>
2019-07-25 23:08:44.974296
<Response [200]>
2019-07-25 23:08:45.572912
<Response [200]>
2019-07-25 23:08:46.182776
<Response [200]>
2019-07-25 23:08:47.066419
<Response [200]>
2019-07-25 23:08:48.864949
<Response [200]>
2019-07-25 23:08:49.879117
<Response [200]>
2019-07-25 23:08:50.769806
<Response [200]>
2019-07-25 23:08:51.688834
<Response [200]>
2019-07-25 23:08:52.478842
<Response [200]>
2019-07-25 23:08:53.276572
<Response [200]>
2019-07-25 23:08:54.301091
<Response [200]>
2019-07-25 23:08:55.086349
<Response [200]>
2019-07-25 23:08:55.999419
<Response [200]>
2019-07-25 23:08:56.895014
<Response [200]>
2019-07-25 23:08:57.695815
<Response [200]>
2019-07-25 23:08:58.493253
<Response [200]>
2019-07-25 23:08:59.591541
<Response [200]>
2019-07-25 23:0

In [47]:
from sqlalchemy import create_engine


driver = 'mysql+pymysql:'
user = 'root'
password = '19693903'
ip = '127.0.0.1'
database = 'project_4'

In [48]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)


In [116]:
def save_to_sql_clean():
    for key, value in dict_airpt_df.items():
        
        value = value.astype({'schd_arri_time': 'datetime64[ns]','schd_depart_time':'datetime64[ns]'})
        value = value[['airln_name', 'airln_code','type', 'codeshared', 'status', 'icaonumb', 'flight','depart_terminal',\
      'depart_gate', 'depart_airpt' ,'schd_depart_time','depart_delay','arri_airpt','schd_arri_time',\
      'arri_delay']]
        value['depart_delay'] = value['depart_delay'].fillna(0)
        value['arri_delay'] = value['arri_delay'].fillna(0)
        value = value.astype({'depart_delay': int,'arri_delay':int})
        value = value.drop_duplicates(subset = ['flight','depart_terminal','schd_depart_time', 'depart_delay', \
                              'schd_arri_time', 'arri_delay'], keep='first')
        
        value.to_sql('flights',engine, if_exists='append', index=False)

In [65]:
save_to_sql_clean()